<a href="https://colab.research.google.com/github/A110ha/Python/blob/master/Practic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from abc import ABC, abstractmethod
import logging
from datetime import datetime
import json


# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("rental_service.log"),
        logging.StreamHandler()
    ]
)


# Базовый класс MusicalInstrument
class MusicalInstrument(ABC):
    registry = {}  # Реестр для подклассов

    def __init__(self, instrument_id: int, name: str, condition: str, daily_rate: float, is_available: bool = True):
        self._instrument_id = instrument_id
        self._name = name
        self._condition = condition
        self._daily_rate = daily_rate
        self._is_available = is_available

    @property
    def instrument_id(self) -> int:
        return self._instrument_id

    @property
    def name(self) -> str:
        return self._name

    @property
    def condition(self) -> str:
        return self._condition

    @property
    def daily_rate(self) -> float:
        return self._daily_rate

    @daily_rate.setter
    def daily_rate(self, value: float):
        if value < 0:
            raise InvalidInstrumentError("Стоимость аренды не может быть отрицательной.")
        self._daily_rate = value

    @property
    def is_available(self) -> bool:
        return self._is_available

    @is_available.setter
    def is_available(self, value: bool):
        self._is_available = value

    @abstractmethod
    def calculate_rental_cost(self, days: int) -> float:
        pass

    def __str__(self) -> str:
        return f"Инструмент: {self.name}, Состояние: {self.condition}, Доступность: {'Да' if self.is_available else 'Нет'}"

    @staticmethod
    def register_instrument(name: str, cls):
        """Регистрация подкласса в реестре."""
        MusicalInstrument.registry[name.lower()] = cls

    @staticmethod
    def get_instrument_class(name: str):
        """Получение класса инструмента по имени."""
        return MusicalInstrument.registry.get(name.lower())


# Миксин для сериализации
class SerializableMixin:
    def to_dict(self):
        return {
            'instrument_id': self.instrument_id,
            'name': self.name,
            'condition': self.condition,
            'daily_rate': self.daily_rate,
            'is_available': self.is_available
        }

    @classmethod
    def from_dict(cls, data):
        return cls(
            instrument_id=data['instrument_id'],
            name=data['name'],
            condition=data['condition'],
            daily_rate=data['daily_rate'],
            is_available=data.get('is_available', True)
        )


# Подклассы для конкретных инструментов
class Guitar(MusicalInstrument, SerializableMixin):
    def __init__(self, instrument_id: int, name: str, condition: str, daily_rate: float, number_of_strings: int):
        super().__init__(instrument_id, name, condition, daily_rate)
        self._number_of_strings = number_of_strings

    @property
    def number_of_strings(self) -> int:
        return self._number_of_strings

    def calculate_rental_cost(self, days: int) -> float:
        base_cost = days * self.daily_rate
        if days > 7:
            return base_cost * 0.9  # Скидка 10% при аренде более 7 дней
        return base_cost

    def __str__(self) -> str:
        return f"{super().__str__()}, Струн: {self.number_of_strings}"


class Piano(MusicalInstrument, SerializableMixin):
    def __init__(self, instrument_id: int, name: str, condition: str, daily_rate: float, key_count: int):
        super().__init__(instrument_id, name, condition, daily_rate)
        self._key_count = key_count

    @property
    def key_count(self) -> int:
        return self._key_count

    def calculate_rental_cost(self, days: int) -> float:
        base_cost = days * self.daily_rate
        if days > 30:
            return base_cost * 0.85  # Скидка 15% при аренде более 30 дней
        return base_cost

    def __str__(self) -> str:
        return f"{super().__str__()}, Клавиш: {self.key_count}"


class Violin(MusicalInstrument, SerializableMixin):
    def __init__(self, instrument_id: int, name: str, condition: str, daily_rate: float, bow_included: bool):
        super().__init__(instrument_id, name, condition, daily_rate)
        self._bow_included = bow_included

    @property
    def bow_included(self) -> bool:
        return self._bow_included

    def calculate_rental_cost(self, days: int) -> float:
        base_cost = days * self.daily_rate
        if self.bow_included:
            return base_cost * 1.1  # +10% если смычок включен
        return base_cost

    def __str__(self) -> str:
        return f"{super().__str__()}, Смычок: {'Да' if self.bow_included else 'Нет'}"


# Регистрация подклассов вручную
MusicalInstrument.register_instrument('guitar', Guitar)
MusicalInstrument.register_instrument('piano', Piano)
MusicalInstrument.register_instrument('violin', Violin)


# Фабричный метод для создания инструментов
class InstrumentFactory:
    @staticmethod
    def create_instrument(instrument_type: str, **kwargs):
        cls = MusicalInstrument.get_instrument_class(instrument_type)
        if cls is None:
            raise InvalidInstrumentError(f"Тип инструмента '{instrument_type}' не поддерживается.")
        return cls(**kwargs)


# Исключения
class InvalidInstrumentError(Exception):
    pass


class PermissionDeniedError(Exception):
    pass


class RentalNotFoundError(Exception):
    pass


# Пример использования
if __name__ == "__main__":
    # Проверка реестра
    print("Реестр инструментов:", list(MusicalInstrument.registry.keys()))

    # Создание инструментов через фабрику
    guitar = InstrumentFactory.create_instrument(
        'guitar',
        instrument_id=1,
        name="Fender",
        condition="Отличное",
        daily_rate=50,
        number_of_strings=6
    )
    piano = InstrumentFactory.create_instrument(
        'piano',
        instrument_id=2,
        name="Yamaha",
        condition="Хорошее",
        daily_rate=100,
        key_count=88
    )

    print(guitar)
    print(piano)

Реестр инструментов: ['guitar', 'piano', 'violin']
Инструмент: Fender, Состояние: Отличное, Доступность: Да, Струн: 6
Инструмент: Yamaha, Состояние: Хорошее, Доступность: Да, Клавиш: 88
